In [2]:
!pip3 install requests 

# Скачиваем мемы 

In [4]:
import requests

url = 'http://books.toscrape.com/'

res = requests.get(url)
res

<Response [200]>

In [6]:
url = 'http://books.toscrape.com/jfeahiogeoierg'
res = requests.get(url)
res

<Response [404]>

In [7]:
url = 'https://knowyourmeme.com/memes'
res = requests.get(url)
res

<Response [403]>

In [8]:
res.headers

{'Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 13 Nov 2020 06:55:44 GMT', 'Content-Type': 'text/html', 'Transfer-Encoding': 'chunked', 'Connection': 'close', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip'}

In [9]:
from fake_useragent import UserAgent
UserAgent().chrome

'Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1467.0 Safari/537.36'

In [10]:
response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

In [11]:
from bs4 import BeautifulSoup

# распарсили страничку в дерево 
tree = BeautifulSoup(response.content, 'html.parser')

In [16]:
tree.head.text

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nConfirmed Entries | Know Your Meme\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

In [17]:
tree.head.text.strip()

'Confirmed Entries | Know Your Meme'

In [22]:
table = tree.find('table', {'class': 'entry_list'})
items = table.find_all('a', {'class': 'photo'})
items[0]

<a class="photo" href="/memes/hoo-hah"><img alt="Hoo Hah" data-src="https://i.kym-cdn.com/entries/icons/medium/000/018/001/maGU9vZ.png" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Hoo Hah"/> <div class="entry-labels"> </div> </a>

In [23]:
items[0].text

'   '

In [24]:
items[0].get("href")

'/memes/hoo-hah'

In [25]:
items[0].img

<img alt="Hoo Hah" data-src="https://i.kym-cdn.com/entries/icons/medium/000/018/001/maGU9vZ.png" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Hoo Hah"/>

In [26]:
items[0].img.get('alt')

'Hoo Hah'

In [31]:
def get_hrefs(p):
    url = f'https://knowyourmeme.com/memes/page/{p}'
    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
    tree = BeautifulSoup(response.content, 'html.parser')
    
    table = tree.find('table', {'class': 'entry_list'})
    items = table.find_all('a', {'class': 'photo'})
    infa = {
        'name': [x.img.get('alt') for x in items],
        'href': [x.get("href") for x in items],
    }
    return infa 

In [32]:
from tqdm import tqdm

infa = {'name': [], 'href': []}

for i in tqdm(range(10)):
    cur = get_hrefs(i)
    
    infa['name'].extend(cur['name'])
    infa['href'].extend(cur['href'])

100%|██████████| 10/10 [00:22<00:00,  2.27s/it]


In [33]:
import pandas as pd
df = pd.DataFrame(infa)
df.head()

,name,href
0,Hoo Hah,/memes/hoo-hah
1,Yoshikage Kira anime character in a purple suit,/memes/my-name-is-yoshikage-kira
2,I Worked On This Story for a Year...,/memes/i-worked-on-this-story-for-a-year
3,Argentina Is White,/memes/argentina-is-white
4,I Feel Fantastic,/memes/i-feel-fantastic


In [34]:
df.to_csv('memes_data.csv', sep='\t', index=None)

# Скачиваем табличку с сайта ЦБ

In [35]:
resp = requests.get('https://cbr.ru/currency_base/daily/')
tree = BeautifulSoup(resp.content, 'html.parser')

In [37]:
# нашли табличку
table = tree.find_all('table', {'class' : 'data'})[0]

In [39]:
# распарсили её
df = pd.read_html(str(table), header=None)[0]
df.head()

,Цифр. код,Букв. код,Единиц,Валюта,Курс
0,36,AUD,1,Австралийский доллар,559622
1,944,AZN,1,Азербайджанский манат,453883
2,51,AMD,100,Армянских драмов,155863
3,933,BYN,1,Белорусский рубль,300549
4,975,BGN,1,Болгарский лев,464771


# API Вконтакте

In [41]:
with open('secret.txt', 'r') as f:
    token = f.readlines()
token = ''.join(token)

In [42]:
version = '5.103' 

In [43]:
method = 'users.get'
parameters = 'user_ids=6045249'

url = 'https://api.vk.com/method/' + method + '?' + parameters + '&v=' + version + '&access_token=' + token

response = requests.get(url) 
response.json()

{'response': [{'first_name': 'Филипп',
   'id': 6045249,
   'last_name': 'Ульянкин',
   'can_access_closed': True,
   'is_closed': False}]}

In [45]:
def vk_download(method, parameters):
    url = 'https://api.vk.com/method/' + method + '?' + parameters + '&v=' + version + '&access_token=' + token
    response = requests.get(url) 
    return response.json()['response']

vk_download('users.get', 'user_ids=6045249')

[{'first_name': 'Филипп',
  'id': 6045249,
  'last_name': 'Ульянкин',
  'can_access_closed': True,
  'is_closed': False}]

In [52]:
inf = vk_download('users.get', 'user_ids=6045249,1,100,151317520,mylibenliska&fields=bdate,about')
df = pd.DataFrame(inf)
df

,first_name,id,last_name,can_access_closed,is_closed,bdate,about
0,Филипп,6045249,Ульянкин,True,False,3.10.1994,
1,Павел,1,Дуров,True,False,10.10.1984,NaN
2,Администрация ВКонтакте,100,,True,False,NaN,NaN
3,Татьяна,151317520,Шелованова,True,False,23.3,NaN
4,Elisaveta,26611242,Churpita,True,False,21.9,


In [54]:
inf[0]

{'first_name': 'Филипп',
 'id': 6045249,
 'last_name': 'Ульянкин',
 'can_access_closed': True,
 'is_closed': False,
 'bdate': '3.10.1994',
 'about': ''}

# Selenium

In [55]:
from selenium import webdriver
driver = webdriver.Chrome('./chromedriver')

In [58]:
ref = 'http://google.com'
driver.get(ref)

In [59]:
stroka = driver.find_element_by_name("q")
stroka.click()

In [60]:
stroka.send_keys('Вконтакте')

In [61]:
button = driver.find_element_by_name('btnK')
button.click()

In [62]:
bs = BeautifulSoup(driver.page_source)

dirty_hrefs = bs.find_all('h3',attrs={'class':'r'})
clean_hrefs = [href.a['href'] for href in dirty_hrefs]
clean_hrefs

['https://vk.com/join',
 'https://vk.com/blog',
 'https://vk.com/login',
 'https://vk.com/search',
 'https://vk.com/page-777107_28406709',
 'https://vk.com/vkteam']

In [63]:
driver.close()